In [43]:
import pandas as pd
df = pd.read_csv(f'data_storage/stock_price_data/Energy/APA.csv')

In [44]:
df['Date'] = pd.to_datetime(df['Date'])

In [45]:
df[df['Date'] == pd.to_datetime("07/01/2024")].index[0]

106

In [46]:
df.sort_values(by="Date").reset_index()

,index,Date,Close/Last,Volume,Open,High,Low
0,1257,2019-12-02,$19.54,22790570,$20.00,$20.42,$18.93
1,1256,2019-12-03,$18.38,15187360,$19.25,$19.30,$18.33
2,1255,2019-12-04,$18.70,10408040,$18.61,$19.16,$18.5003
3,1254,2019-12-05,$18.58,7503783,$18.78,$18.9824,$18.44
4,1253,2019-12-06,$19.99,10085920,$18.50,$20.17,$18.33
...,...,...,...,...,...,...,...
1253,4,2024-11-22,$22.77,12821090,$22.29,$22.99,$22.265
1254,3,2024-11-25,$22.67,7440440,$22.79,$22.98,$22.435
1255,2,2024-11-26,$22.52,4680421,$22.76,$22.835,$22.245
1256,1,2024-11-27,$22.52,4679593,$22.54,$23.14,$22.36


In [103]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by="Date").reset_index()
date = pd.to_datetime("07/01/2024")
while len(df[df['Date'] == date]) == 0:
    date += pd.DateOffset(days=1)
index = df[df['Date']==pd.to_datetime(date)].index[0]
output_index = df[df['Date']==pd.to_datetime(date)+pd.DateOffset(days=30)].index[0]


In [113]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np


features = ['Open', 'Low', 'Volume']

num_data = df[['Open','High','Low']].to_numpy().astype(str)
num_data = np.char.replace(num_data, '$', '')


output_var = df[['Close/Last']].to_numpy().astype(str)
output_var = np.char.replace(output_var, '$', '')
output_var = output_var.astype(float)

scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(num_data)
feature_transform= pd.DataFrame(columns=features, data=feature_transform, index=df.index)

X_train, X_test = feature_transform[:index], feature_transform[index:]
offset = len(output_var[:output_index]) - len(output_var[:index])
y_train, y_test = output_var[offset:output_index], output_var[output_index:]

In [ ]:
{
    'Consumer_Discretionary': ['CTAS', 'SBUX'], 
    'Real Estate': ['LAMR'], 
    'Consumer_Staples': ['MDLZ'], 
    'Industrials': ['HON']
}


array([[25.36],
       [25.69],
       [25.64],
       ...,
       [31.21],
       [30.78],
       [31.03]])

In [115]:
trainX =np.array(X_train)
testX =np.array(X_test)
X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])

In [116]:
X_train.shape

(1151, 1, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.utils import plot_model

# Define early stopping
early_stop = EarlyStopping(
    monitor='loss',   # Monitor validation loss
    patience=5,           # Stop after 3 epochs with no improvement
    restore_best_weights=True  # Roll back to best model
)

lstm = Sequential()
lstm.add(LSTM(32, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
plot_model(lstm, show_shapes=True, show_layer_names=True)
history=lstm.fit(X_train, y_train, epochs=30, batch_size=8, verbose=1, shuffle=False, callbacks=[early_stop])


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/30
144/144 [==============================] - 2s 2ms/step - loss: 918.3852
Epoch 2/30
144/144 [==============================] - 0s 2ms/step - loss: 791.4161
Epoch 3/30
144/144 [==============================] - 0s 2ms/step - loss: 542.4244
Epoch 4/30
144/144 [==============================] - 0s 2ms/step - loss: 271.7872
Epoch 5/30
144/144 [==============================] - 0s 2ms/step - loss: 101.5590
Epoch 6/30
144/144 [==============================] - 0s 2ms/step - loss: 39.8326
Epoch 7/30
144/144 [==============================] - 0s 2ms/step - loss: 27.2368
Epoch 8/30
144/144 [==============================] - 0s 2ms/step - loss: 25.7645
Epoch 9/30
144/144 [==============================] - 0s 2ms/step - loss: 25.6804
Epoch 10/30
144/144 [==============================] - 0s 2ms/step - loss: 25.6936
Epoch 11/30
144/144 [====

In [126]:

articles = pd.read_csv("data_storage/sentiment_scores/aggregate_sentiment_final.csv")
articles['stock']

0        AMZN
1        AMZN
2        AMZN
3        AMZN
4        AMZN
         ... 
16645      TW
16646      TW
16647      TW
16648      TW
16649      TW
Name: stock, Length: 16650, dtype: object

In [127]:
articles[(articles['stock'] == 'AMZN') & (articles['source'] == 'news')]

,sector,stock,text,source,source_title,source_url,date,summarized_text,sentiment,sentiment_updated
112,Consumer_Discretionary,AMZN,Amazon stock with a split-adjusted price will ...,news,Amazon stock to trade at split-adjusted price ...,https://www.financialexpress.com/investing-abr...,2022-06-06 12:00:15,Amazon stock with a split-adjusted price will ...,-0.102,0.9709
113,Consumer_Discretionary,AMZN,The Dow Jones Industrial Average edged 0.3% hi...,news,Dow Jones Up Despite Jobs Growth Miss; Amazon ...,https://worldnewsera.com/news/finance/banking/...,2022-04-01 21:54:24,Amazon (AMZN) was firm despite a recent unioni...,-0.117,0.8442
114,Consumer_Discretionary,AMZN,"If you are holding Amazon stocks, there's goin...",news,Amazon Stock Split: Get 19 additional shares f...,https://www.financialexpress.com/investing-abr...,2022-05-30 12:23:01,The article discusses Amazon's stock (AMZN) in...,-0.023,0.8885
115,Consumer_Discretionary,AMZN,Jordan Novet / CNBC: Amazon stock closed down ...,news,"Amazon Stock Closed Down ~8%, Meta Fell ~7%, A...",https://worldnewsera.com/news/tech-tips/amazon...,2022-05-05 22:58:18,Amazon stock closed down ~8% as a tech selloff...,-0.605,0.3400
116,Consumer_Discretionary,AMZN,"Apple AMZN Above, I marked the 1600 and 1300 l...",news,"AMZN Back To 1300, Nasdaq Back To 7000 For NAS...",https://worldnewsera.com/news/finance/stock-ma...,2022-04-26 17:52:32,"Yes, the article talks about AMZN. \n\nThe art...",0.229,-0.1027
...,...,...,...,...,...,...,...,...,...,...
678,Consumer_Discretionary,AMZN,SOME people have made a good living by reselli...,news,Where to buy Amazon return pallets,https://www.thesun.co.uk/fabulous/17398122/whe...,2024-08-21 16:04:32,"Yes, the article talks about AMZN. \n\nThe art...",0.063,0.9042
679,Consumer_Discretionary,AMZN,Amazon founder Jeff Bezos is planning to sell ...,news,Jeff Bezos to sell $5bn in Amazon shares,https://www.aljazeera.com/news/2024/7/4/jeff-b...,2024-07-04 09:53:56,Amazon founder Jeff Bezos is planning to sell ...,0.246,0.9678
680,Consumer_Discretionary,AMZN,Amazon founder Jeff Bezos is planning to sell ...,news,Jeff Bezos to sell $5B worth of Amazon shares ...,https://nypost.com/2024/07/03/business/jeff-be...,2024-07-03 20:15:52,Amazon founder Jeff Bezos is planning to sell ...,-0.272,0.9738
681,Consumer_Discretionary,AMZN,Dividend cuts and suspensions can often come w...,news,"Intel Suspended Its Dividend, and This Stock C...",https://finance.yahoo.com/news/intel-suspended...,2024-08-29 12:15:00,"Yes, the article talks about AMZN. It mentions...",-0.164,0.8625
